# **Project Name**    -  **Brain Tumor MRI Image Classification**



##### **Project Type**    - **Classification**
##### **Contribution**    - **Individual**
##### **Name              -** **Alwin Shaji**

# **Project Summary**

This project focuses on developing a deep learning-based system to classify brain MRI images into four categories: glioma, meningioma, pituitary tumor, and no tumor. The primary objective is to create an automated solution that can assist radiologists by reducing diagnostic time and improving efficiency. MRI scans often contain subtle variations that require significant expertise to interpret, and a model capable of accurately predicting tumor type can help streamline workflows, especially in resource-limited settings. By leveraging image preprocessing, data augmentation, and advanced CNN architectures, the system aims to handle variability in image quality and patient positioning.

The planned approach involves building two types of models: a custom CNN to establish a baseline and a fine-tuned ResNet50 model for higher accuracy using transfer learning. Class imbalance will be addressed through class weights, and training will be monitored using early stopping, learning rate reduction, and checkpointing to prevent overfitting and optimize resource usage. Performance evaluation will include accuracy, precision, recall, F1 score, and confusion matrix analysis, ensuring that the model’s reliability is thoroughly assessed.

Finally, explainability is a key part of the plan. Visualization tools such as Grad-CAM will be integrated to highlight regions influencing predictions, building trust in the model’s decision-making process. The best-performing model will be saved in .h5 format for future deployment, and an interactive demo using Streamlit is envisioned to showcase real-time predictions. This approach not only targets technical accuracy but also prioritizes usability, interpretability, and real-world relevance in medical imaging applications.



# **GitHub Link -**

Provide your GitHub Link here.

# **Problem Statement**


This project aims to develop a deep learning-based solution for classifying brain MRI images into multiple categories according to tumor type. It involves building a custom CNN model from scratch and enhancing performance through transfer learning using pretrained models. The project also includes deploying a user-friendly Streamlit web application to enable real-time tumor type predictions from uploaded MRI images.


# **General Guidelines** : -  

1.   Well-structured, formatted, and commented code is required.
2.   Exception Handling, Production Grade Code & Deployment Ready Code will be a plus. Those students will be awarded some additional credits.
     
     The additional credits will have advantages over other students during Star Student selection.
       
             [ Note: - Deployment Ready Code is defined as, the whole .ipynb notebook should be executable in one go
                       without a single error logged. ]

3.   Each and every logic should have proper comments.
4. You may add as many number of charts you want. Make Sure for each and every chart the following format should be answered.
        

```
# Chart visualization code
```
            

*   Why did you pick the specific chart?
*   What is/are the insight(s) found from the chart?
* Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

5. You have to create at least 15 logical & meaningful charts having important insights.


[ Hints : - Do the Vizualization in  a structured way while following "UBM" Rule.

U - Univariate Analysis,

B - Bivariate Analysis (Numerical - Categorical, Numerical - Numerical, Categorical - Categorical)

M - Multivariate Analysis
 ]





6. You may add more ml algorithms for model creation. Make sure for each and every algorithm, the following format should be answered.


*   Explain the ML Model used and it's performance using Evaluation metric Score Chart.


*   Cross- Validation & Hyperparameter Tuning

*   Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

*   Explain each evaluation metric's indication towards business and the business impact pf the ML model used.




















# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


### Dataset Loading

In [ ]:
import zipfile
import os

# Path to your uploaded ZIP file
zip_path = '/content/drive/MyDrive/Tumor.zip'

# Destination to unzip
extract_path = '/content/tumor_data'

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check folders
os.listdir(extract_path)



### Dataset First View

In [ ]:

# Path to extracted dataset
dataset_path = '/content/tumor_data/Tumour'

# List everything in the dataset path
all_items = os.listdir(dataset_path)
print("All items in dataset:", all_items)

# Only keep folders (ignore files like README)
categories = [item for item in all_items if os.path.isdir(os.path.join(dataset_path, item))]
print("\nValid category folders:", categories)

# Count number of images in each category folder
for category in categories:
    folder = os.path.join(dataset_path, category)
    num_images = sum([len(files) for _, _, files in os.walk(folder)])
    print(f"{category}: {num_images} images")


### Dataset Information

In [ ]:

from PIL import Image

# Paths to data splits
base_path = '/content/tumor_data/Tumour'
train_path = os.path.join(base_path, 'train')
valid_path = os.path.join(base_path, 'valid')
test_path  = os.path.join(base_path, 'test')

# Count images function (unchanged)
def count_images(folder_path):
    total = 0
    for class_folder in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_folder)
        if os.path.isdir(class_path):
            total += len(os.listdir(class_path))
    return total

# Image counts
num_train = count_images(train_path)
num_valid = count_images(valid_path)
num_test  = count_images(test_path)

print(f"🧾 Number of training images: {num_train}")
print(f"🧾 Number of validation images: {num_valid}")
print(f"🧾 Number of test images: {num_test}")

# Filter only class folders (ignore .csv file)
class_names = sorted([f for f in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, f))])
print(f"\n📚 Classes ({len(class_names)}): {class_names}")

# View sample image size from the first actual class folder
first_class = class_names[0]
first_img_path = os.path.join(train_path, first_class, os.listdir(os.path.join(train_path, first_class))[0])
img = Image.open(first_img_path)
print(f"\n🖼 Sample image size: {img.size}")


In [ ]:

def count_images_in_each_class(folder_path):
    print(f"📁 Folder: {folder_path}")
    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if os.path.isdir(class_path):
            image_count = len(os.listdir(class_path))
            print(f"🔹 {class_name}: {image_count} images")

# Check training set
count_images_in_each_class('/content/tumor_data/Tumour/train')

# Check validation set
count_images_in_each_class('/content/tumor_data/Tumour/valid')

# Check test set
count_images_in_each_class('/content/tumor_data/Tumour/test')


### What did you know about your dataset?

The dataset is organized into three main folders: train, valid, and test, each containing subfolders for the four classes — glioma, meningioma, no_tumor, and pituitary. There are 1695 training images, 502 validation images, and 246 test images. The images are MRI scans with a sample resolution of 640×640 pixels. Since the dataset is already pre-split and well-structured by folder names, no manual label handling or data splitting is required. We'll resize the images during preprocessing to make them suitable for CNN input.




## ***2. Understanding Your Images***

Each sample in the dataset is a brain MRI image stored in JPEG or PNG format. The average resolution is 640×640 pixels. Images are grayscale or RGB scans taken from different angles and positions. The four classes — glioma, meningioma, pituitary, and no_tumor — correspond to visual patterns in the brain indicating the presence or absence of tumors. These differences may appear as irregular shapes, dense masses, or abnormal textures in specific regions. Understanding these patterns is key for the CNN model to learn how to classify the images accurately.

## 3. ***Data Wrangling***

There is no traditional data wrangling step required since the dataset is already clean and organized. The images are sorted into folders representing their respective classes. Instead of wrangling, we'll focus on preparing the images using preprocessing techniques such as resizing, normalization, and augmentation to make them suitable for deep learning models.




## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

# Make plots a little cleaner
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.0)

# Paths
BASE_DIR   = '/content/tumor_data/Tumour'
TRAIN_DIR  = os.path.join(BASE_DIR, 'train')
VALID_DIR  = os.path.join(BASE_DIR, 'valid')
TEST_DIR   = os.path.join(BASE_DIR, 'test')

IMG_SIZE   = (224, 224)
BATCH_SIZE = 32


In [ ]:
def count_images_in_dir(dir_path):
    counts = {}
    for cls in sorted(os.listdir(dir_path)):
        cls_path = os.path.join(dir_path, cls)
        if os.path.isdir(cls_path):
            counts[cls] = len(os.listdir(cls_path))
    return counts

train_counts = count_images_in_dir(TRAIN_DIR)

plt.figure(figsize=(6,4))
plt.bar(train_counts.keys(), train_counts.values(), color='red')
plt.title('Training Set Class Distribution')
plt.ylabel('Number of Images')
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

print("Training counts:", train_counts)


##### 1. Why did you pick the specific chart?

This bar chart shows the number of images available for each brain tumor category in the training dataset.

##### 2. What is/are the insight(s) found from the chart?

It helps identify class imbalance. For example, if the glioma class has significantly more samples than no_tumor, the model might become biased toward glioma predictions.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

By spotting imbalance early, I applied class weighting during training to ensure fairness across classes, leading to more reliable diagnostic predictions.

#### Chart - 2

In [ ]:
train_total = sum(count_images_in_dir(TRAIN_DIR).values())
valid_total = sum(count_images_in_dir(VALID_DIR).values())
test_total  = sum(count_images_in_dir(TEST_DIR).values())

splits = ['Train', 'Valid', 'Test']
vals   = [train_total, valid_total, test_total]

plt.figure(figsize=(5,4))
plt.bar(splits, vals, color='green')
plt.title('Dataset Split Sizes')
plt.ylabel('Image Count')
plt.tight_layout()
plt.show()

print(f"Train: {train_total}, Valid: {valid_total}, Test: {test_total}")


##### 1. Why did you pick the specific chart?

This chart visualizes how the dataset is divided into training, validation, and test sets.

##### 2. What is/are the insight(s) found from the chart?

It confirms whether the data split maintains a healthy balance for model training, validation tuning, and unbiased performance testing (e.g., 70-20-10 ratio).

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Ensuring a proper split avoids overfitting and guarantees reliable generalization in real-world tumor classification scenarios.

#### Chart - 3

In [ ]:
import random
from PIL import Image
import matplotlib.pyplot as plt
import os
import math

def show_samples_per_class(root_dir, n_per_class=3, max_rows=3):
    # Get class names
    classes = [cls for cls in sorted(os.listdir(root_dir)) if os.path.isdir(os.path.join(root_dir, cls))]
    if not classes:
        print("No classes found in:", root_dir)
        return

    # Limit rows to max_rows
    classes = classes[:max_rows]

    # Set figure size dynamically
    plt.figure(figsize=(n_per_class * 3, max_rows * 3))
    plt.suptitle("Sample Images from Each Class", fontsize=16, fontweight='bold')

    idx = 1
    for row, cls in enumerate(classes):
        cls_dir = os.path.join(root_dir, cls)
        imgs = [os.path.join(cls_dir, f) for f in os.listdir(cls_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        if len(imgs) == 0:
            continue

        # Pick random samples for the class
        samples = random.sample(imgs, min(n_per_class, len(imgs)))

        for j, img_path in enumerate(samples):
            img = Image.open(img_path)
            plt.subplot(max_rows, n_per_class, idx)
            plt.imshow(img)
            plt.axis('off')

            # Add heading above first image of the row
            if j == 0:
                plt.ylabel(cls, fontsize=12, fontweight='bold', rotation=0, labelpad=30)

            idx += 1

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust to make space for title
    plt.show()

# ✅ Example usage:
TRAIN_DIR = '/content/tumor_data/Tumour/train'
show_samples_per_class(TRAIN_DIR, n_per_class=3, max_rows=3)


##### 1. Why did you pick the specific chart?

This bar chart shows the number of images available for each tumor class (Glioma, Meningioma, Pituitary, No Tumor) in the dataset.

##### 2. What is/are the insight(s) found from the chart?

It highlights any class imbalance, which can negatively impact model performance by biasing predictions toward majority classes.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Detecting imbalance early allows corrective measures like class weighting or augmentation, improving diagnostic accuracy across all tumor types.



#### Chart - 4

In [ ]:
# Build a small augmentation generator (match training settings)
aug_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Grab a batch
preview_gen = aug_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=1,
    class_mode='categorical',
    shuffle=True
)

# Take one image and show augmented variants
x_batch, y_batch = next(preview_gen)
img = x_batch[0]

n_aug = 6
plt.figure(figsize=(10,2))
for i in range(n_aug):
    aug_x, _ = next(preview_gen)
    plt.subplot(1, n_aug, i+1)
    plt.imshow(aug_x[0])
    plt.axis('off')
plt.suptitle('Augmentation Preview (random transforms)', y=0.95)
plt.tight_layout()
plt.show()


##### 1. Why did you pick the specific chart?

This chart shows different variations of the same image after applying transformations like rotation, zoom, and flips, demonstrating how data augmentation increases diversity in the training set to improve model generalization.

##### 2. What is/are the insight(s) found from the chart?

Data augmentation significantly increases variability in the training data, which helps reduce overfitting by exposing the model to different orientations and lighting conditions.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

By improving generalization, the model becomes more reliable in real-world scenarios, reducing misclassifications and improving diagnostic accuracy for brain tumor detection.

#### Chart - 5

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
model = tf.keras.models.load_model('/content/drive/MyDrive/final_trained_model.h5', compile=False)


# ----------------- 2. Test Data Generator -----------------
TEST_PATH = '/content/tumor_data/Tumour/test'  # Change path if needed
IMG_SIZE = (224, 224)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# ----------------- 3. Predictions -----------------
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# ----------------- 4. Classification Report -----------------
print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

# ----------------- 5. Confusion Matrix -----------------
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Brain Tumor Classification")
plt.show()

# ----------------- 6. Overall Accuracy -----------------
accuracy = (y_pred == y_true).mean() * 100
print(f"Overall Accuracy: {accuracy:.2f}%")



In [ ]:
# Compute and plot class-wise accuracy
def class_wise_accuracy(y_true, y_pred, class_labels):
    accs = []
    for i, cls in enumerate(class_labels):
        idx = np.where(y_true == i)[0]
        cls_acc = (y_pred[idx] == y_true[idx]).mean() if len(idx) else np.nan

        accs.append(cls_acc * 100)
    plt.figure(figsize=(6,4))
    plt.bar(class_labels, accs, color='blue')
    plt.ylabel('Accuracy (%)')
    plt.title('Class-wise Accuracy')
    plt.xticks(rotation=15)
    plt.ylim(0, 100)
    plt.tight_layout()
    plt.show()
    return accs

# ✅ Call after predictions
class_wise_accuracy(y_true, y_pred, class_labels)



##### 1. Why did you pick the specific chart?

The class-wise accuracy chart is critical because overall accuracy can hide weaknesses in specific categories. For medical applications like brain tumor detection, misclassification in one class (e.g., no tumor vs. glioma) can have severe consequences. This chart ensures transparency about how the model behaves for each class.



##### 2. What is/are the insight(s) found from the chart?

The chart revealed that some classes, such as pituitary and meningioma, achieved higher accuracy, while others like glioma performed relatively lower. This indicates that the model might need more representative samples or targeted augmentation for certain classes.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Focusing on improving accuracy for underperforming tumor types reduces diagnostic errors, leading to more reliable medical predictions. This can enhance trust in the system for clinical use and minimize risks associated with misdiagnosis, ultimately improving patient safety and healthcare outcomes.



#### Chart - 6 - Correlation Heatmap


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Assuming y_pred_probs from model.predict(test_generator)
# and class_labels already defined

# Create DataFrame of probabilities
probs_df = pd.DataFrame(y_pred_probs, columns=class_labels)

# Compute correlation matrix
corr_matrix = probs_df.corr()

# Plot heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of Predicted Class Probabilities')
plt.show()



##### 1. Why did you pick the specific chart?

Shows relationships between classes based on model predictions. If two tumor classes have high correlation, the model often confuses them.

##### 2. What is/are the insight(s) found from the chart?

If glioma and meningioma show strong positive correlation, it explains misclassifications and indicates they share visual similarities.

#### Chart - 7 - Pair Plot

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

# 1. Reduce prediction probabilities to 3 PCA components
pca = PCA(n_components=3)
reduced = pca.fit_transform(y_pred_probs)

# 2. Create DataFrame for visualization
df = pd.DataFrame(reduced, columns=['PC1', 'PC2', 'PC3'])
df['Class'] = [class_labels[i] for i in test_generator.classes]

# 3. Plot Pairplot
sns.pairplot(df, hue='Class', vars=['PC1', 'PC2', 'PC3'], palette='husl')
plt.suptitle("PCA Pairplot of Prediction Probabilities", y=1.02)
plt.show()


##### 1. Why did you pick the specific chart?

Helps visualize high-dimensional CNN features in 2D/3D space for class separability.

##### 2. What is/are the insight(s) found from the chart?

Observed overlapping clusters between glioma and meningioma, explaining why these classes confused the model.

## ***5. Hypothesis Testing***

Hypothesis testing was not included in this project because the primary goal was to build a predictive model for accurate brain tumor classification, not to infer population parameters or validate statistical assumptions. The evaluation focused on performance metrics such as accuracy, precision, recall, F1-score, and confusion matrix, which are more relevant for deep learning models. However, hypothesis testing could be explored in future work to statistically compare different models and validate performance improvements.


## ***6. Feature Engineering & Data Pre-processing***

Traditional preprocessing steps like handling missing values, outliers, or categorical encoding are not required in image-based projects. Since the dataset is composed of labeled images organized in folders, the model learns directly from visual patterns. Preprocessing here refers to image resizing, normalization, and augmentation instead of tabular data cleaning.




### 1. Data Transformation

#### Do you think that your data needs to be transformed? If yes, which transformation have you used. Explain Why?

Yes, the data needs to be transformed using augmentation techniques like rotation, flipping, zoom, and brightness adjustments. These transformations help simulate real-world variability in MRI scans and prevent the model from overfitting to limited patterns. By artificially increasing the diversity of the training data, the model learns to generalize better on unseen images.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image augmentation applied only to training data
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True
)


### 2. Data Scaling

In [ ]:
# Normalization generator (applied to ALL splits: train, valid, test)
scaling_datagen = ImageDataGenerator(rescale=1./255)


##### Which method have you used to scale you data and why?

We used min-max scaling by setting rescale=1./255 in the ImageDataGenerator, which converts pixel values from the range [0, 255] to [0, 1]. This normalization helps the model train faster, reduces numerical instability, and ensures consistency across images. It’s a standard and effective scaling method for deep learning tasks involving image data.



### 3. Dimesionality Reduction

##### Do you think that dimensionality reduction is needed? Explain Why?

Yes, dimensionality reduction is needed in this project. The original MRI images are 640×640 pixels, which are large and computationally expensive to process. By resizing them to 224×224 pixels, we reduce memory usage and training time while still preserving the essential features needed for tumor classification. This size is also compatible with most pretrained CNN architectures like ResNet and VGG.

In [ ]:
# Redefine the dataset paths if they were lost
train_path = '/content/tumor_data/Tumour/train'
valid_path = '/content/tumor_data/Tumour/valid'
test_path  = '/content/tumor_data/Tumour/test'

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),  # This is dimensionality reduction!
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

##### Which dimensionality reduction technique have you used and why? (If dimensionality reduction done on dataset.)


We used **image resizing** as the dimensionality reduction technique by setting the target size to **224×224 pixels**. This was done using the `target_size` parameter in `ImageDataGenerator`. It helps reduce computational load while keeping important visual features intact for accurate classification.


### 4. Data Splitting

No, additional data splitting is not needed.

The dataset already comes pre-divided into three folders: train, valid, and test. Each folder contains images organized by class, which makes it ready to be directly used with ImageDataGenerator without any manual splitting.



### 5. Handling Imbalanced Dataset

##### Do you think the dataset is imbalanced? Explain Why.

The dataset shows a moderate class imbalance, with glioma having the highest number of images and no_tumor the least. To address this, we used data augmentation during training to increase the diversity of underrepresented classes. Additionally, we will apply class_weight during model training to ensure that the model doesn't become biased toward the majority classes.


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get class indices from the train_generator
class_indices = train_generator.class_indices  # e.g., {'glioma': 0, 'meningioma': 1, ...}
class_labels = list(class_indices.values())   # [0, 1, 2, 3]

# Compute weights
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)

# Convert to dictionary format expected by Keras
class_weights = dict(zip(class_labels, class_weights_array))

print("📊 Computed Class Weights:", class_weights)


##### What technique did you use to handle the imbalance dataset and why? (If needed to be balanced)

 Class weighting technique was used to handle the moderate imbalance in the dataset. This approach assigns higher importance (loss penalty) to underrepresented classes during training. It ensures the model treats all classes fairly, even when some have fewer images, improving overall classification performance and reducing bias.



## ***7. ML Model Implementation***

### ML Model - 1

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, GlobalAveragePooling2D,
    Dense, Dropout, BatchNormalization
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal
from sklearn.utils.class_weight import compute_class_weight

# Paths / Config
TRAIN_PATH = '/content/tumor_data/Tumour/train'
VALID_PATH = '/content/tumor_data/Tumour/valid'

BEST_MODEL_PATH = '/content/drive/MyDrive/cnn_last_epoch.h5'
LAST_MODEL_PATH = '/content/drive/MyDrive/improved_cnn_best_valacc.h5'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Skip switch: set True to skip retraining
SKIP_TRAINING = True   # <-- flip to False to train


# Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,
    zoom_range=0.2,
    brightness_range=(0.8, 1.2),
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    VALID_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


# Class Weights (imbalance handling)
class_weights_arr = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights_arr))
print("Class weights:", class_weights)


# Build CNN
def build_model():
    return Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same',
               kernel_initializer=HeNormal(), input_shape=(224, 224, 3)),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=HeNormal()),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=HeNormal()),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=HeNormal()),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.4),

        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=HeNormal()),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.5),

        # Head
        GlobalAveragePooling2D(),
        Dense(128, activation='relu', kernel_initializer=HeNormal()),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 classes
    ])


model = build_model()

# Compile
model.compile(
    optimizer=Adam(learning_rate=2e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


# Callbacks
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=6,
    restore_best_weights=True,
    mode='max',
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

checkpoint = ModelCheckpoint(
    BEST_MODEL_PATH,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)



# Train (guarded skip)
if SKIP_TRAINING:
    if os.path.exists(BEST_MODEL_PATH):
        print("⏩ Skipping training (already trained, best model exists at: "
              f"{BEST_MODEL_PATH}).")
    else:
        print("⚠ SKIP_TRAINING=True but no saved model found. "
              "Set SKIP_TRAINING=False to train.")
else:
    history = model.fit(
        train_generator,
        validation_data=valid_generator,
        epochs=EPOCHS,
        class_weight=class_weights,
        callbacks=[early_stop, reduce_lr, checkpoint],
        verbose=1
    )
    # Save last epoch model
    model.save(LAST_MODEL_PATH)
    print("✅ Training complete.")
    print("Saved best (val_acc) to:", BEST_MODEL_PATH)
    print("Saved last epoch model to:", LAST_MODEL_PATH)


#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

Built a Custom Convolutional Neural Network (CNN) from scratch using Keras. The model has 3 convolutional blocks for feature extraction, followed by dense layers for classification. It was trained to classify brain MRI images into four categories: glioma, meningioma, pituitary tumor and no tumor.



In [ ]:

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score


# Paths
model_path = '/content/drive/MyDrive/cnn_last_epoch.h5'
test_path = '/content/tumor_data/Tumour/test'
IMG_SIZE = (224, 224)

# Load Model
model = load_model(model_path)

# Test Data Generator
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Predictions
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Classification Report
print("\n📊 CNN Model Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix - CNN Model")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.tight_layout()
plt.show()

# Accuracy & Loss Chart

def plot_history(history):
    plt.figure(figsize=(12,5))

    # Accuracy
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"\n✅ Overall Accuracy: {accuracy * 100:.2f}%")


#### 2. Cross- Validation & Hyperparameter Tuning

##### Which hyperparameter optimization technique have you used and why?

I used **manual tuning combined with callbacks like EarlyStopping and ReduceLROnPlateau** to optimize learning rate and prevent overfitting. This approach was chosen because it’s faster and more practical for large CNN models compared to exhaustive automated searches.


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Config
train_path = '/content/tumor_data/Tumour/train'
valid_path = '/content/tumor_data/Tumour/valid'

BEST_MODEL_PATH = '/content/tuned_cnn_best.h5'
FINAL_MODEL_PATH = '/content/drive/MyDrive/tuned_cnn_final.h5'

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 25

# Flip to False to retrain
SKIP_TRAINING = True

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)
valid_gen = valid_datagen.flow_from_directory(
    valid_path, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical'
)

# Model Architecture
def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.3),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.4),

        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')
    ])
    return model

model = build_model()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
checkpoint = ModelCheckpoint(
    BEST_MODEL_PATH,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=8,
    restore_best_weights=True,
    verbose=1
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# Train (skip-aware)
if SKIP_TRAINING:
    if os.path.exists(FINAL_MODEL_PATH) or os.path.exists(BEST_MODEL_PATH):
        print("⏩ Skipping training (already trained; model saved).")
    else:
        print("⚠ SKIP_TRAINING=True but no saved model found. Set SKIP_TRAINING=False to train.")
else:
    history = model.fit(
        train_gen,
        validation_data=valid_gen,
        epochs=EPOCHS,
        callbacks=[checkpoint, early_stop, reduce_lr],
        verbose=1
    )
    # Save final model
    model.save(FINAL_MODEL_PATH)
    print("✅ Model saved as", FINAL_MODEL_PATH)



##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

In [ ]:

# ✅ CNN Evaluation (Test Only)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Paths & Config
BEST_MODEL_PATH = '/content/drive/MyDrive/tuned_cnn_final.h5'
TEST_PATH = '/content/tumor_data/Tumour/test'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32


# 2. Load Model
print("Loading best saved model...")
model = load_model(BEST_MODEL_PATH)


# 3. Prepare Test Data Generator
datagen = ImageDataGenerator(rescale=1./255)

test_gen = datagen.flow_from_directory(
    TEST_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


# 4. Evaluation on Test Set
y_true = test_gen.classes
class_labels = list(test_gen.class_indices.keys())

print("\nPredicting on Test set...")
probs = model.predict(test_gen, verbose=1)
y_pred = np.argmax(probs, axis=1)

accuracy = (y_true == y_pred).mean()
print(f"\n✅ Test Accuracy: {accuracy*100:.2f}%")

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - Test Set")
plt.tight_layout()
plt.show()


### ML Model - 2

In [ ]:
 # Imports & Setup
import os
from collections import Counter
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Paths & Config
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 10

train_path = '/content/tumor_data/Tumour/train'
valid_path = '/content/tumor_data/Tumour/valid'

# Paths for saving
BEST_CHECKPOINT_PATH = '/content/drive/MyDrive/best_model.h5'   # checkpoint
FINAL_MODEL_PATH     = '/content/drive/MyDrive/final_trained_model.h5'  # last-epoch save

# Skip switch: set True to avoid retraining
SKIP_TRAINING = True   # <-- flip to False to retrain


# Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Class Weights
class_counts = Counter(train_generator.classes)
total = sum(class_counts.values())
class_weights = {i: total / (len(class_counts) * count) for i, count in class_counts.items()}
print("Class weights:", class_weights)

#  Skip
if SKIP_TRAINING:
    if os.path.exists(FINAL_MODEL_PATH):
        print(f"⏩ Skipping training. Loading model from: {FINAL_MODEL_PATH}")
        model = load_model(FINAL_MODEL_PATH)
    else:
        print(f"⚠ SKIP_TRAINING=True but model not found at {FINAL_MODEL_PATH}.")
        print("➡ Please set SKIP_TRAINING=False to train the model.")
        raise SystemExit("Stopping execution to avoid retraining.")
else:
    print("✅ Training started...")

    # Build model
    base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))

    # Freeze first 100 layers
    for layer in base_model.layers[:100]:
        layer.trainable = False
    for layer in base_model.layers[100:]:
        layer.trainable = True

    # Custom classification head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    outputs = Dense(4, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    # Compile
    model.compile(optimizer=Adam(learning_rate=1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Callbacks
    early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1)
    checkpoint = ModelCheckpoint(BEST_CHECKPOINT_PATH, monitor='val_loss', save_best_only=True, verbose=1)

    # Train
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=valid_generator,
        callbacks=[early_stop, checkpoint],
        class_weight=class_weights,
        verbose=1
    )

    # Save final model
    model.save(FINAL_MODEL_PATH)
    print(f"✅ Training complete. Best checkpoint: {BEST_CHECKPOINT_PATH}, Final model: {FINAL_MODEL_PATH}")


#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

I used ResNet50, a deep convolutional neural network pretrained on ImageNet, to classify brain MRI images into four categories: glioma, meningioma, pituitary, and no_tumor. ResNet50’s architecture uses residual connections to train deeper networks efficiently, making it ideal for complex image classification tasks.

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# 1. Load the trained ResNet50 model
model = load_model('/content/drive/MyDrive/best_model.h5')

# 2. Define paths
test_path = '/content/tumor_data/Tumour/test'
IMG_SIZE = (224, 224)

# 3. Preprocess test images
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


# 4. Predict on test set
y_pred_probs = model.predict(test_generator, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())


# 5. Compute Overall Accuracy
accuracy = np.mean(y_true == y_pred) * 100
print(f"\n✅ Overall Test Accuracy: {accuracy:.2f}%")

# 6. Classification report
print("\n📊 Classification Report for ResNet50:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

# 7. Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples',
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title(f"ResNet50 - Confusion Matrix ")
plt.tight_layout()
plt.show()


#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)


In [ ]:
import os
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight

# ----------- CONFIG -----------
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
TRAIN_PATH = '/content/tumor_data/Tumour/train'
VALID_PATH = '/content/tumor_data/Tumour/valid'
MODEL_SAVE_PATH = '/content/drive/MyDrive/best_model_finetuned.h5'  # checkpoint + final

# Skip switch: set True to avoid retraining
SKIP_TRAINING = True  # flip to False when you want to train again


# DATA PREPROCESSING
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.15,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
valid_generator = valid_datagen.flow_from_directory(
    VALID_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# CLASS WEIGHTS
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)


# Skip or Train
if SKIP_TRAINING:
    if os.path.exists(MODEL_SAVE_PATH):
        print(f"⏩ Skipping training. Loading saved model from: {MODEL_SAVE_PATH}")
        model = load_model(MODEL_SAVE_PATH)
    else:
        print(f"⚠ SKIP_TRAINING=True but no saved model found at {MODEL_SAVE_PATH}.")
        print("Set SKIP_TRAINING=False to train.")
        raise SystemExit("Stopped to prevent unintended long training.")
else:
    print("✅ Starting fine-tuning...")

    # MODEL BUILDING
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

    # Unfreeze top 50% of layers
    cutoff = int(len(base_model.layers) * 0.5)
    for layer in base_model.layers[:cutoff]:
        layer.trainable = False
    for layer in base_model.layers[cutoff:]:
        layer.trainable = True

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(4, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    # COMPILE
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # CALLBACKS
    early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1)
    checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_loss', save_best_only=True, verbose=1)

    # TRAIN
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=valid_generator,
        callbacks=[early_stop, checkpoint],
        class_weight=class_weights,
        verbose=1
    )

    # Explicit final save (overwrites with final weights — optional)
    model.save(MODEL_SAVE_PATH)
    print(f"✅ Training complete. Model saved to: {MODEL_SAVE_PATH}")


In [ ]:
model.save("final_trained_model.keras")   # creates a .keras zip file

model.export("final_trained_model_savedmodel")   # folder
# or explicit:
# tf.saved_model.save(model, "final_trained_model_savedmodel")



In [ ]:
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Paths
CHECKPOINT_PATH = '/content/drive/MyDrive/best_model_finetuned.h5'   # weights after first 10 epochs
FINAL_MODEL_PATH = '/content/drive/MyDrive/final_trained_model.h5'   # after resume (epoch 20)

# Skip control
SKIP_RESUME = True   # flip to False to continue training

# Resume or Skip
if SKIP_RESUME:
    if os.path.exists(FINAL_MODEL_PATH):
        print(f"⏩ Skipping resume training (final model already saved). Loading {FINAL_MODEL_PATH} for evaluation.")
        model = load_model(FINAL_MODEL_PATH)
    elif os.path.exists(CHECKPOINT_PATH):
        print(f"⏩ Skipping resume; using checkpoint at epoch 10: {CHECKPOINT_PATH}")
        model = load_model(CHECKPOINT_PATH)
    else:
        raise FileNotFoundError(
            "Skip requested but no saved model found. "
            "Set SKIP_RESUME=False to resume training."
        )
else:
    # Load checkpoint from first training phase if needed
    if 'model' not in globals():
        if os.path.exists(CHECKPOINT_PATH):
            print(f"Loading checkpoint from: {CHECKPOINT_PATH}")
            model = load_model(CHECKPOINT_PATH)
        else:
            raise FileNotFoundError(f"Checkpoint not found at {CHECKPOINT_PATH} and no model in memory.")

    # Callbacks for resume phase
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        verbose=1,
        min_lr=1e-6
    )

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    # Resume Training
    history = model.fit(
        train_generator,
        epochs=20,              # total target epochs
        initial_epoch=10,       # resume from 10
        validation_data=valid_generator,
        callbacks=[early_stop, reduce_lr],
        class_weight=class_weights,
        verbose=1
    )

    # Save final trained model
    model.save(FINAL_MODEL_PATH)
    print(f"✅ Resume complete. Final model saved to: {FINAL_MODEL_PATH}")


##### Which hyperparameter optimization technique have you used and why?


I used **ReduceLROnPlateau** as the hyperparameter optimization technique. It dynamically reduces the learning rate when the validation loss plateaus, helping the model converge better and avoid overshooting minima. This improves fine-tuning efficiency and stabilizes training for better accuracy.


##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

In [ ]:
# Imports
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load your trained model from Google Drive or local path
model = load_model('/content/drive/MyDrive/final_trained_model.h5')  # or 'best_model.keras' if you used that format

# Define test data path
test_path = '/content/tumor_data/Tumour/test'
IMG_SIZE = (224, 224)

# Create test data generator
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # Important for matching predictions with labels
)

# Predict
y_probs = model.predict(test_generator)
y_pred = np.argmax(y_probs, axis=1)
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# 📊 Classification Report
print("\n📈 Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# 🔍 Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# 📉 Plot Confusion Matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='YlGnBu', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Brain Tumor Classification")
plt.tight_layout()
plt.show()

# ✅ Overall Accuracy
accuracy = np.sum(y_true == y_pred) / len(y_true)
print(f"\n✅ Overall Test Accuracy: {accuracy * 100:.2f}%")


#### 3. Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

###  Evaluation Metrics & Business Impact

- **Accuracy**: Shows overall correctness. High accuracy builds trust but may be misleading with imbalanced classes.

- **Precision**: Measures how many predicted tumor cases are actually correct. Helps reduce false alarms and unnecessary medical procedures.

- **Recall**: Captures how many actual tumor cases the model detects. Crucial in healthcare — missing a tumor can be fatal.

- **F1-Score**: Balances precision and recall. Indicates a model that’s both reliable and safe for medical deployment.

- **Confusion Matrix**: Reveals specific misclassifications. Helps improve the model for underperforming tumor types.

###  Business Impact

Using a ResNet50-based model improves diagnosis speed, reduces radiologist workload, and enhances early detection — leading to better patient outcomes and increased operational efficiency.


### 2. Which ML model did you choose from the above created models as your final prediction model and why?

I chose the **fine-tuned ResNet50 model** as the final prediction model. It leverages transfer learning from ImageNet, giving it a strong ability to extract complex features compared to custom CNNs. This model achieved better validation accuracy and generalization due to its deep architecture and fine-tuning strategy, making it the most reliable choice for brain tumor classification.


### 3. Explain the model which you have used and the feature importance using any model explainability tool?

I used ResNet50, a deep convolutional neural network with 50 layers, based on residual learning. It introduces skip connections (residual blocks) to solve the vanishing gradient problem, making it effective for training very deep networks. In this project, the base ResNet50 was fine-tuned on brain tumor images, with a custom classification head for four classes: glioma, meningioma, pituitary, and no tumor.

For feature importance, I used Grad-CAM (Gradient-weighted Class Activation Mapping). Grad-CAM highlights the regions in the image that most influenced the model’s prediction by computing the gradient of the output class with respect to the feature maps. This helps visualize which areas of the brain MRI contributed most to classifying a specific tumor type.

## ***8.*** ***Future Work (Optional)***

### 1. Save the best performing ml model for deployment process.


The .h5 format in Keras saves the complete model, including architecture, weights, and optimizer state, making it ready for direct inference and deployment. Unlike joblib or pickle, which are used for scikit-learn models, .h5 is the standard for TensorFlow/Keras. This format works seamlessly in deployment environments such as Flask, FastAPI, or Streamlit. Additionally, it can be easily converted to TensorFlow Lite or ONNX for optimized deployment.

### ***Congrats! Your model is successfully created and ready for deployment on a live server for a real user interaction !!!***

# **Conclusion**

I started this project by preprocessing and augmenting the MRI dataset to reduce overfitting and improve generalization. After experimenting with a custom CNN for baseline results, I moved to transfer learning using ResNet50, combined with class balancing, dropout, and regularization to enhance performance.

The tuned ResNet50 achieved 84.15% validation accuracy, which was significantly better than the earlier models. I evaluated the model using precision, recall, F1-score, and confusion matrices to understand its strengths and weaknesses. To ensure reliability, I also applied Grad-CAM for explainability, confirming that predictions focused on tumor regions. The final model was saved as an H5 file, making it ready for deployment.

Future improvements include adding more data, trying 3D CNN architectures, and validating the model on clinical datasets. This project shows how deep learning can be effectively applied to medical image classification while maintaining explainability and robustness.



### ***Hurrah! You have successfully completed your Deep Learning Capstone Project !!!***